In [1]:
# --------------------------------------------------------------------------
# Generate Results for the SUN Attributes Ranked Training Dataset 
# --------------------------------------------------------------------------
# import pylab as pltss
# from pylab import *
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
import scipy
import scipy.io
import os.path

# -----------------------------------------------------
# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

# -----------------------------------------------------
# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.
MODEL_FILE = 'deploy_sun_ws.prototxt'
PRETRAINED = 'caffe_sun_ws.caffemodel'

import os

# Change this to GPU later
caffe.set_mode_cpu()

net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       mean = np.load('sunours_mean.npy').mean(1).mean(1),
                       channel_swap = (2,1,0),
                       raw_scale = 255,
                       image_dims = (256, 256))

In [2]:
import glob
base_path = '/Users/ashish/vision-project/data/SUN/SUN_WS/training/'
# A = glob.glob(base_path + '*')
# for a in A:
#     y = lables.index(a.split('/')[-1])

In [3]:
for blob in net.blobs:
    print blob, 
    print net.blobs[blob].data.shape

data (400, 3, 227, 227)
conv1 (400, 96, 55, 55)
pool1 (400, 96, 27, 27)
norm1 (400, 96, 27, 27)
conv2 (400, 256, 27, 27)
pool2 (400, 256, 13, 13)
norm2 (400, 256, 13, 13)
conv3 (400, 384, 13, 13)
conv4 (400, 384, 13, 13)
conv5 (400, 256, 13, 13)
pool5 (400, 256, 6, 6)
fc6 (400, 4096)
fc7 (400, 4096)
fc8 (400, 42)
prob (400, 42)


In [4]:
imPaths = glob.glob(base_path + '*/*.jpg')

In [11]:
cat = [i.split('/')[-2] for i in imPaths]

In [15]:
labels = ['vegetation',
    'shrubbery',
    'foliage',
    'leaves',
    'shingles',
    'concrete',
    'metal',
    'paper',
    'wood',
    'vinyl',
    'rubber',
    'cloth',
    'sand',
    'rock',
    'dirt',
    'marble',
    'glass',
    'waves',
    'running_water',
    'still_water',
    'snow',
    'natural_light',
    'direct_sun',
    'electric',
    'aged',
    'glossy',
    'matte',
    'moist',
    'dry',
    'dirty',
    'rusty',
    'warm',
    'cold',
    'natural',
    'man_made',
    'open_area',
    'far_away_horizon',
    'rugged_scene',
    'symmetrical',
    'cluttered_space',
    'scary',
    'soothing']

In [24]:
y = [labels.index(cat_) + 1 for cat_ in cat]
y = np.asarray(y)


In [25]:
np.save('labels.npy', y)

In [ ]:
feat_vecs = np.zeros((1, 4096))
batch_size = net.blobs['data'].data.shape[0]
num_im = len(imPaths)

iter_ = 0
max_iter = len(imPaths) / batch_size
while imPaths:
    iter_ += 1
    print('iteration', iter_, 'out of', max_iter)
    ims = []
    images = []
    
    for i in range(batch_size):
        if imPaths:
            ims.append(imPaths.pop())
        else:
            break

    for im in ims:
        image = caffe.io.load_image(im)
        images.append(image)
        
    # predict takes any number of images, and formats them for the Caffe net automatically
    prediction = net.predict(images)
    feat = net.blobs['fc7'].data
    feat_vecs = np.vstack([feat_vecs, feat])

feat_vecs = feat_vecs[1:num_im+1, :]

In [ ]:
print(feat_vecs.shape)

In [ ]:
np.save('feat_vecs.npy', feat_vecs)